In [ ]:
!pip install librosa

from google.colab import drive
drive.mount('/content/drive')

import librosa
import numpy as np
import os
import glob
import logging
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [ ]:
log_file = '/content/drive/MyDrive/Colab Notebooks/GenreRecog/extraction.log'
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    force=True
)

dataset_path = "/content/drive/MyDrive/Colab Notebooks/GenreRecog/Data/genres_original"

def extract_mfcc(file_path, n_mfcc=13, sr=22050):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc = np.mean(mfcc, axis=1)

        mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)

        logging.info(f"Extracted MFCCs from {file_path}")
        return mfcc
    except Exception as e:
        logging.error(f"Error extracting MFCCs from {file_path}: {e}")
        return None

def load_dataset(dataset_path):
    genres = os.listdir(dataset_path)
    data = []
    labels = []

    logging.info(f"Loading dataset from {dataset_path}")
    for genre in genres:
        genre_path = os.path.join(dataset_path, genre)
        audio_files = glob.glob(os.path.join(genre_path, '*.wav'))
        logging.info(f"Processing genre: {genre}")

        for audio_file in audio_files:
            mfcc = extract_mfcc(audio_file)
            if mfcc is not None:
                data.append(mfcc)
                labels.append(genre)
                logging.info(f"Loaded {len(data)} audio files.")
            else:
                logging.warning(f"Skipping file {audio_file} due to MFCC extraction error.")

    logging.info(f"Dataset loading complete. Total samples: {len(data)}")
    return np.array(data), np.array(labels)

X, y = load_dataset(dataset_path)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
logging.info("Labels encoded successfully.")

In [ ]:
!pip install torch torchvision

log_file='/content/drive/MyDrive/Colab Notebooks/GenreRecog/model.log'
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    force=True
)

class MusicGenreNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MusicGenreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

input_size = 13
num_classes = len(np.unique(y_encoded))

logging.info(f"Input size: {input_size}, Number of classes: {num_classes}")

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

logging.info(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
logging.info(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

model = MusicGenreNN(input_dim=input_size, num_classes=num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

logging.info(f"Model architecture: {model}")
logging.info(f"Optimizer: {optimizer}")

In [ ]:
log_file='/content/drive/MyDrive/Colab Notebooks/GenreRecog/training.log'
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    force=True
)
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    logging.info(f"Starting epoch {epoch + 1}/{num_epochs}")

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if (i + 1) % 100 == 0:
            logging.info(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / (i + 1):.4f}")

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    logging.info(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/GenreRecog/music_genre_model.pth')
logging.info("Model saved successfully")

logging.info("Training complete")

In [ ]:
log_file='/content/drive/MyDrive/Colab Notebooks/GenreRecog/evaluation.log'
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    force=True
)

model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")

report = classification_report(all_labels, all_predictions, target_names=label_encoder.classes_)
logging.info(f"Classification Report:\n{report}")
print(report)


logging.info(f"Accuracy: {accuracy:.2f}%")
logging.info(f"Classification Report:\n{report}")


In [ ]:
#neccesary to create blank log file to force previous log file to show in Drive

import logging
import os

log_file='/content/drive/MyDrive/Colab Notebooks/GenreRecog/blank.log'
logging.basicConfig(
    filename=log_file,
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    force=True
)